In [1]:
import yfinance as yf
from datetime import datetime
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
import statsmodels as smt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
Visa = yf.download('V',start_date ,end_date,index=False)

[*********************100%***********************]  1 of 1 completed


In [2]:
Visa.reset_index(inplace=True)
Visa

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-03-01,149.460007,149.929993,148.380005,149.470001,146.724579,7016800
1,2019-03-04,150.070007,150.190002,146.949997,147.960007,145.242355,6979700
2,2019-03-05,148.000000,149.289993,147.830002,147.949997,145.232544,9678600
3,2019-03-06,148.000000,148.520004,147.419998,147.809998,145.095108,8186400
4,2019-03-07,147.259995,147.809998,146.039993,146.830002,144.133087,8761700
...,...,...,...,...,...,...,...
268,2020-03-24,144.839996,155.250000,143.580002,154.529999,152.619736,24488300
269,2020-03-25,159.320007,168.179993,153.580002,161.779999,159.780121,20619600
270,2020-03-26,162.080002,169.529999,161.789993,168.880005,166.792374,17062900
271,2020-03-27,161.080002,167.100006,158.149994,161.559998,159.562836,14950700


In [3]:
# df = pd.read_csv('F-F_Research_Data_Factors_daily.csv',skiprows=4)
# df

In [4]:
start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
JP = yf.download('JPM',start_date ,end_date,index=False)

[*********************100%***********************]  1 of 1 completed


In [5]:
start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
Master_card = yf.download('MA',start_date ,end_date,index=False)

[*********************100%***********************]  1 of 1 completed


In [6]:
start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
Bofa = yf.download('BAC',start_date ,end_date,index=False)

[*********************100%***********************]  1 of 1 completed


In [7]:
# df.rename(columns = {'Unnamed: 0':'Date'},inplace=True)

In [8]:
# df=df[:-1]

In [9]:
# df['Date']=pd.to_datetime(df['Date'])

In [10]:
# df

In [11]:
Visa.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [12]:
df_vjp = Visa.merge(JP, how='left', on='Date')
df_vjp = df_vjp.rename(columns=lambda x: x.replace('_x', ''))
df_vjp = df_vjp[['Date','Adj Close','Adj Close_y']]
df_vjp.rename(columns={'Adj Close':'Adj_Close_Visa','Adj Close_y':'Adj_Close_JP'},inplace=True)
df_vjp


df_vmc = Visa.merge(Master_card, how='left', on='Date')
df_vmc = df_vmc.rename(columns=lambda x: x.replace('_x', ''))
df_vmc = df_vmc[['Date','Adj Close','Adj Close_y']]
df_vmc.rename(columns={'Adj Close':'Adj_Close_Visa','Adj Close_y':'Adj_Close_Master'},inplace=True)
df_vmc

df_vba = Visa.merge(Bofa, how='left', on='Date')
df_vba = df_vba.rename(columns=lambda x: x.replace('_x', ''))
df_vba = df_vba[['Date','Adj Close','Adj Close_y']]
df_vba.rename(columns={'Adj Close':'Adj_Close_Visa','Adj Close_y':'Adj_Close_BOfA'},inplace=True)
df_vba


df_final = df_vmc.merge(df_vjp, how='left', on=['Date','Adj_Close_Visa'])
df_final = df_final.merge(df_vba, how='left', on=['Date','Adj_Close_Visa'])
df_final

,Date,Adj_Close_Visa,Adj_Close_Master,Adj_Close_JP,Adj_Close_BOfA
0,2019-03-01,146.724579,223.466217,94.824356,27.388264
1,2019-03-04,145.242355,222.522202,94.606438,27.126621
2,2019-03-05,145.232544,222.453369,94.533783,27.173342
3,2019-03-06,145.095108,220.732498,94.179657,26.949081
4,2019-03-07,144.133087,218.706818,93.498650,26.678091
...,...,...,...,...,...
268,2020-03-24,152.619736,234.327652,82.663338,20.123718
269,2020-03-25,159.780121,253.513138,85.748146,20.190702
270,2020-03-26,166.792374,260.135620,91.721436,21.740889
271,2020-03-27,159.562836,244.785263,85.187263,20.669159


In [13]:
df_final['Visa_y-1']=df_final['Adj_Close_Visa'].shift(periods=1)
df_final=df_final[1:]
df_final

,Date,Adj_Close_Visa,Adj_Close_Master,Adj_Close_JP,Adj_Close_BOfA,Visa_y-1
1,2019-03-04,145.242355,222.522202,94.606438,27.126621,146.724579
2,2019-03-05,145.232544,222.453369,94.533783,27.173342,145.242355
3,2019-03-06,145.095108,220.732498,94.179657,26.949081,145.232544
4,2019-03-07,144.133087,218.706818,93.498650,26.678091,145.095108
5,2019-03-08,144.643555,218.175781,93.534958,26.771536,144.133087
...,...,...,...,...,...,...
268,2020-03-24,152.619736,234.327652,82.663338,20.123718,134.062027
269,2020-03-25,159.780121,253.513138,85.748146,20.190702,152.619736
270,2020-03-26,166.792374,260.135620,91.721436,21.740889,159.780121
271,2020-03-27,159.562836,244.785263,85.187263,20.669159,166.792374


In [14]:
X = df_final[[ 'Adj_Close_Master', 'Adj_Close_JP','Adj_Close_BOfA']]
y = df_final['Adj_Close_Visa']

# X = sm.add_constant(X)

# x = StandardScaler().fit_transform(X)
pca = PCA(n_components=3)
components = pca.fit_transform(X)


df_components = pd.DataFrame(data=components, columns=['pc1', 'pc2','pc3'])
df_components.index += 1 

In [15]:
# df_final_1 = X.merge(df_final['Visa_y-1'],how='left')
# df_final_1 = pd.merge(df_components, df_final['Visa_y-1'], left_index=True, right_index=True)
# df_final_1

df_final_1 = pd.concat([df_components, df_final['Visa_y-1']], axis=1)
df_final_1

,pc1,pc2,pc3,Visa_y-1
1,48.304485,-5.527637,-1.284517,146.724579
2,48.391615,-5.503888,-1.348108,145.242355
3,50.130987,-5.789108,-1.173956,145.232544
4,52.278661,-5.891208,-1.041052,145.095108
5,52.747324,-6.153971,-1.099015,144.133087
...,...,...,...,...
268,42.485031,11.504037,1.380628,134.062027
269,23.615829,16.145357,1.502247,152.619736
270,15.127330,13.001557,1.551668,159.780121
271,31.840834,13.147888,1.212786,166.792374


In [16]:
y 

1      145.242355
2      145.232544
3      145.095108
4      144.133087
5      144.643555
          ...    
268    152.619736
269    159.780121
270    166.792374
271    159.562836
272    163.523254
Name: Adj_Close_Visa, Length: 272, dtype: float64

In [17]:
df_final['Visa_y-1']

1      146.724579
2      145.242355
3      145.232544
4      145.095108
5      144.133087
          ...    
268    134.062027
269    152.619736
270    159.780121
271    166.792374
272    159.562836
Name: Visa_y-1, Length: 272, dtype: float64

In [18]:
ff_model = sm.OLS(y, df_final_1).fit()
print(ff_model.summary())

                                 OLS Regression Results                                
Dep. Variable:         Adj_Close_Visa   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          1.546e+05
Date:                Fri, 22 Apr 2022   Prob (F-statistic):                        0.00
Time:                        15:27:06   Log-Likelihood:                         -735.44
No. Observations:                 272   AIC:                                      1479.
Df Residuals:                     268   BIC:                                      1493.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [19]:
intercept, b1, b2, b3 = ff_model.params

In [20]:
intercept, b1, b2, b3 

(-0.011267140577729689,
 -0.0177281655040675,
 -0.6379614876769739,
 0.9999470641515764)